In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
from costs import *
from lab_helpers import *
from plots_lab4 import *
from my_helpers import *
import pandas as pd
import matplotlib.pyplot as plt

## To Do Graphs
- PCA Decomposition + Classification
- Correlation
- Changing loss for different algorithms
- ???

## To Do tasks
- Manuel: implement K-fold to choose degree, change split data , deal with outliers, adding log colm (?)
- Gabbo: ?
- Marco: ?

## Load the training data into feature matrix, class labels, and event ids:

In [49]:
DATA_TRAIN_PATH = '../data/train.csv'
y_or, tX_or, ids = load_csv_data(DATA_TRAIN_PATH)

In [50]:
y = y_or
tX = tX_or

In [51]:
#for logistic
y = y_for_logistic(y)

## Data preprocessing

In [52]:
x_tr_tot, x_te_tot, y_tr_tot, y_te_tot = split_data(y,tX,0.8,1)

In [53]:
y_tr, tX_tr, indexes_tr, means_tr, std_tr = preprocessing(y_tr_tot, x_tr_tot)

In [54]:
y_te, tX_te, indexes_te, means_te, std_te = preprocessing(y_te_tot, x_te_tot)

In [68]:
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    N = tx.shape[0]
    first = np.sum(np.log(1+np.exp(tx.dot(w))))
    second = y.T.dot(tx.dot(w))
    #print(first, second)
    return first - second

In [69]:
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return tx.T.dot(sigmoid(tx.dot(w))-y)

In [35]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    grad = calculate_gradient(y, tx, w)
    w = w - gamma * grad
    loss = calculate_loss(y, tx, w)
    return loss, w

In [40]:
def logistic_regression_gradient_descent(y, x):
    # init parameters
    max_iter = 100
    threshold = 1e-8
    gamma = 0.00001
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = 0.1 * np.ones((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        # log info
        if iter % 10 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return loss, w

In [41]:
list_loss = []
list_w = []
for jet in range(0,8):
    tX_tr_poly = build_poly(tX_tr[jet], 2)
    loss, w = logistic_regression_gradient_descent(y_tr[jet], tX_tr_poly)
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, loss=0.8243033141192283
Current iteration=10, loss=0.2198441377775378
Current iteration=20, loss=0.19707586215677328
Current iteration=30, loss=0.19084509480491382
Current iteration=40, loss=0.18732902443561852
Current iteration=50, loss=0.1847501020111033
Current iteration=60, loss=0.18264869630211839
Current iteration=70, loss=0.18085300082414002
Current iteration=80, loss=0.18037106878743375
Current iteration=90, loss=0.1810236014589492
Current iteration=0, loss=6.815345924472263


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


Current iteration=10, loss=inf
Current iteration=20, loss=inf
Current iteration=30, loss=inf
Current iteration=40, loss=inf
Current iteration=50, loss=inf
Current iteration=60, loss=inf
Current iteration=70, loss=inf
Current iteration=80, loss=inf
Current iteration=90, loss=inf
Current iteration=0, loss=0.4389772048594174
Current iteration=10, loss=0.3505732762398289


KeyboardInterrupt: 

In [ ]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

In [65]:
def logistic_regression_sub_gradient_descent(y, x):
    # init parameters
    max_iter = 10000
    threshold = 1e-8
    gamma = 0.05
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.ones((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        for yn, xn in batch_iter(y, tx, 1):
            loss, w = learning_by_gradient_descent(yn, xn, w, gamma)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return loss, w

In [70]:
list_loss = []
list_w = []
for jet in range(0,8):
    tX_tr_poly = build_poly(tX_tr[jet], 7)
    loss, w = logistic_regression_sub_gradient_descent(y_tr[jet], tX_tr_poly)
    list_loss.append(loss)
    list_w.append(w)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


Current iteration=0, loss=inf
Current iteration=0, loss=50.33417848349883
Current iteration=0, loss=0.0
Current iteration=0, loss=0.0
Current iteration=0, loss=0.0
Current iteration=0, loss=0.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


Current iteration=0, loss=26.50210206937815
Current iteration=0, loss=0.0


In [71]:
#cell for logistic
tX_te_poly = list()
for t in tX_te:
    tX_te_poly.append(build_poly(t, 7))
y_pred = build_predictions(tX_te_poly, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.62908


In [23]:
def calculate_hessian(y, tx, w):
    """return the hessian of the loss function."""
    N = tx.shape[0]
    S = np.zeros((N,N))
    for i in range(N):
        sigma = sigmoid(tx[i,:].T.dot(w))
        S[i,i] = sigma*(1-sigma)
    return tx.T.dot(S).dot(tx)

In [24]:
def logistic_regression(y, tx, w):
    """return the loss, gradient, and hessian."""
    return calculate_loss(y, tx, w), calculate_gradient(y, tx, w), calculate_hessian(y, tx, w)

In [25]:
def learning_by_newton_method(y, tx, w):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    # mettiamo il gamma???????
    gamma = 0.001
    loss, gradient, hessian = logistic_regression(y, tx, w)
    w = w - gamma * np.linalg.inv(hessian).dot(gradient)
    return loss, w

In [28]:
def logistic_regression_newton_method_stochastic(y, x):
    # init parameters
    max_iter = 100
    threshold = 1e-8
    lambda_ = 0.1
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.random.randn(tx.shape[1])
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        for yn, xn in batch_iter(y, tx, 10):
            loss, w = learning_by_newton_method(yn, xn, w)
            #print (xn.shape)
            #print(calculate_hessian(yn, xn, w).shape)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return loss, w

In [29]:
list_loss = []
list_w = []
for jet in range(0,8):
    loss, w = logistic_regression_newton_method_stochastic(y_tr[jet], tX_tr[jet])
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, the loss=12.390456798923466


LinAlgError: Singular matrix

In [56]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient, and hessian."""
    N = tx.shape[0]
    D = tx.shape[1]
    loss = calculate_loss(y, tx, w) + lambda_ / 2 * np.linalg.norm(w)
    gradient = calculate_gradient(y, tx, w) + lambda_ * w
    
    return loss, gradient

In [57]:
def penalized_logistic_regression_newton_method(y, tx, w, lambda_):
    """return the loss, gradient, and hessian."""
    N = tx.shape[0]
    D = tx.shape[1]
    loss = calculate_loss(y, tx, w) + lambda_ / 2 * np.linalg.norm(w)
    gradient = calculate_gradient(y, tx, w) + lambda_ * w
    hessian = calculate_hessian(y, tx, w) + N * np.identity(D)
    return loss, gradient, hessian

In [31]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    #loss, gradient, hessian = penalized_logistic_regression(y, tx, w, lambda_)
    loss, gradient = penalized_logistic_regression(y, tx, w, lambda_)
    w = w - gamma * gradient
    return loss, w

In [32]:
def logistic_regression_penalized_gradient_descent(y, x):
    # init parameters
    max_iter = 10000
    gamma = 0.00001
    lambda_ = 0.1
    threshold = 1e-8
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.zeros((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 1000 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return loss, w

In [205]:
list_loss = []
list_w = []
for jet in range(0,8):
    loss, w = logistic_regression_penalized_gradient_descent(y_tr[jet], tX_tr[jet])
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, loss=14466.674805466619
Current iteration=1000, loss=3596.1595805480833
Current iteration=2000, loss=3590.400230578218
Current iteration=3000, loss=3588.004053035492
Current iteration=4000, loss=3586.0540170462436
Current iteration=5000, loss=3584.374003250526
Current iteration=6000, loss=3582.911314632176
Current iteration=7000, loss=3581.6332421478874
Current iteration=8000, loss=3580.51377168331
Current iteration=9000, loss=3579.5308840230578
Current iteration=0, loss=40931.72730642589
Current iteration=1000, loss=28898.054993920876
Current iteration=2000, loss=28841.118146807963
Current iteration=3000, loss=28827.039627332924
Current iteration=4000, loss=28817.220521561856
Current iteration=5000, loss=28808.083030179223
Current iteration=6000, loss=28799.214736122674
Current iteration=7000, loss=28790.539796435227
Current iteration=8000, loss=28782.025698634723
Current iteration=9000, loss=28773.650047570416
Current iteration=0, loss=4160.962524901352
Current i

IndexError: list index out of range

In [ ]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

In [76]:
def logistic_regression_penalized_stochastic_gradient_descent(y, x):
    # init parameters
    max_iter = 10000
    threshold = 1e-8
    gamma = 0.01
    lambda_ = 0
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.ones((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        for yn, xn in batch_iter(y, tx, 1):
            loss, w = learning_by_penalized_gradient(yn, xn, w, gamma, lambda_)
        # log info
        if iter % (max_iter/10) == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return loss, w

In [77]:
list_loss = []
list_w = []
for jet in range(0,8):
    loss, w = logistic_regression_penalized_stochastic_gradient_descent(y_tr[jet], tX_tr[jet])
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, loss=0.7555256486804144
Current iteration=1000, loss=0.012238656768706734
Current iteration=2000, loss=0.005820496995088685
Current iteration=3000, loss=0.06012810338893344
Current iteration=4000, loss=0.018972298564030352
Current iteration=5000, loss=0.019456661049023653
Current iteration=6000, loss=0.011670927039103267
Current iteration=7000, loss=0.018894005733628563
Current iteration=8000, loss=0.041473146185138324
Current iteration=9000, loss=0.09097548189700955
Current iteration=0, loss=0.5918680985802579
Current iteration=1000, loss=2.3283782529487027
Current iteration=2000, loss=0.6542968402672625
Current iteration=3000, loss=0.39883805705496556
Current iteration=4000, loss=0.1358530061016728
Current iteration=5000, loss=0.4711703187564267
Current iteration=6000, loss=0.4511016383396661
Current iteration=7000, loss=0.5550902637393551
Current iteration=8000, loss=0.2776371935707709
Current iteration=9000, loss=0.08006908984995012
Current iteration=0, loss=0.

In [78]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.74536


In [38]:
def learning_by_penalized_newton_method(y, tx, w, gamma, lambda_):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    loss, gradient, hessian = penalized_logistic_regression_newton_method(y, tx, w, lambda_)
    w = w - gamma * np.linalg.inv(hessian).dot(gradient)
    return loss, w

In [46]:
def logistic_regression_penalized_stochastic_newton_method(y, x):
    # init parameters
    max_iter = 1000
    threshold = 1e-8
    gamma = 0.01
    lambda_ = 0.00001
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0],)), x]
    tx = x
    w = np.ones((tx.shape[1],))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        for yn, xn in batch_iter(y, tx, 10):
            loss, w = learning_by_penalized_newton_method(yn, xn, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return loss, w

In [47]:
list_loss = []
list_w = []
for jet in range(0,8):
    loss, w = logistic_regression_penalized_stochastic_newton_method(y_tr[jet], tX_tr[jet])
    list_loss.append(loss)
    list_w.append(w)

Current iteration=0, loss=61.163505199107995
Current iteration=100, loss=4.495607085124813
Current iteration=200, loss=6.008608529547024
Current iteration=300, loss=4.340153518666623
Current iteration=400, loss=4.4714533021184195
Current iteration=500, loss=3.2827953213383276
Current iteration=600, loss=4.082524019965291
Current iteration=700, loss=3.4382800842043415
Current iteration=800, loss=0.2766099375315662
Current iteration=900, loss=2.697715091604993
Current iteration=0, loss=26.663539751757227
Current iteration=100, loss=7.9030892799918
Current iteration=200, loss=5.204082968295155
Current iteration=300, loss=9.722157288835515
Current iteration=400, loss=7.466749452980252
Current iteration=500, loss=14.622667541512756
Current iteration=600, loss=4.063807413810574
Current iteration=700, loss=8.542093994134419
Current iteration=800, loss=3.1285467370552444
Current iteration=900, loss=7.330349870474667
Current iteration=0, loss=27.957074090843985
Current iteration=100, loss=21.33

In [48]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w, logistic=True) 
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.66538


## Generate predictions and save ouput in csv format for submission:

#### Use Accuracy and F1 Score on our classification:

In [183]:
#cell for logistic
y_pred = build_predictions(tX_te, indexes_te, list_w,logistic=True) 


In [184]:
y_pred

array([0., 0., 1., ..., 0., 0., 0.])

In [185]:
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.7516


#### Build submission

In [186]:
DATA_TEST_PATH = 'test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test_new, indexes_test_new, means_test, stds_test = preprocessing([], tX_test, test=True) #same function as train,

In [190]:
#cell for logistic
y_pred_test = build_predictions(tX_test_new, indexes_test_new, list_w, logistic=True)
OUTPUT_PATH = 'submission.csv'
create_csv_submission(ids_test, y_pred_test, OUTPUT_PATH, logistic=True) #logistic flags convert [0, 1] to [-1, 1]

In [12]:
#we will need something like this

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(tX_test_s)
plt.scatter(principalComponents[:,0], principalComponents[:,1])

NameError: name 'tX_test_s' is not defined